In [ ]:
import pandas as pd
iso_df = pd.read_csv(filepath_or_buffer='/kaggle/input/country-mapping-iso-continent-region/continents2.csv',
                    usecols=['name', 'alpha-3', 'region'],).rename(columns={'name': 'Countries'})
iso_df.head()

In [ ]:
country_fixes = \
{'Ant.& Barb.' : 'Antigua and Barbuda',
 'Bosnia & Herz.' : 'Bosnia And Herzegovina',
 'Brunei' : 'Brunei Darussalam',
 'Burma' : 'Myanmar',
 'C.A. Republic' : 'Central African Republic',
 'Cape Verde' : 'Cabo Verde',
 'Czechia' : 'Czech Republic',
 'DR Congo' : 'Congo (Democratic Republic Of The)',
 'Domin. Rep.' : 'Dominican Republic',
 'Eq. Guinea' : 'Equatorial Guinea',
 'G.-Bissau' : 'Guinea Bissau',
 'Ivory Coast' : "Côte D'Ivoire",
 'Micronesia' : 'Micronesia (Federated States of)',
 'North Macedonia' : 'Macedonia',
 'Papua N.G.': 'Papua New Guinea',
 'R. of Congo' : 'Congo',
 'S.T.&Principe' : 'Sao Tome and Principe',
 'Solomon Isl.' : 'Solomon Islands',
 'St. Vincent & ...' : 'Saint Vincent and the Grenadines',
 'Swaziland' : 'Eswatini',
 'Tr.&Tobago' : 'Trinidad and Tobago',
 'UA Emirates' : 'United Arab Emirates',
 'UK' : 'United Kingdom',
 'USA' : 'United States'}

In [ ]:
inflation_df = pd.read_csv(filepath_or_buffer='/kaggle/input/imf-forecast-dataset/Inflation forecast.csv')
inflation_df['Countries'] = inflation_df['Countries'].replace(to_replace=country_fixes)
inflation_df.head()

In [ ]:
df = inflation_df.merge(right=iso_df, on='Countries', how='inner')
df.info()

In [ ]:
from plotly.express import histogram
histogram(data_frame=df, x='Inflation forecast, 2023', hover_name='Countries', color='region')

There is a lot of skew in our data because of a few countries with forecast inflation above 35%.

In [ ]:
from plotly.express import choropleth
choropleth(data_frame=df, locations='alpha-3', color='Inflation forecast, 2023')